In [1]:
!pwd

/home/tucker/sabbatical/predict_bert_embeddings/char_lm/token_start_pred


In [1]:
import plotly
import plotly.graph_objs as go
import torch
from torch.utils.data import DataLoader
from experiment import Net, InitDataset, CreateModel, LoadCheckpoint
from yaml import safe_load
import torch.nn as nn
exphash = '561fdb2f9'

exp = 'results/%s' % exphash
with open(exp) as f:
  h = safe_load(f.read())
h = h['exp_info']  


FileNotFoundError: [Errno 2] No such file or directory: 'results/561fdb2f9'

In [2]:
h['dataset_split'][0] = .1
ds = InitDataset(h, 'cuda')

NameError: name 'h' is not defined

In [3]:
model = CreateModel(h['hyperparameters'])
LoadCheckpoint(model, 'models/%s'%exphash)

model = model.cuda()


[12:10:21] Loaded model: <All keys matched successfully>


In [4]:
a = torch.load('imdb_tok96_char_encoded384charidx2.pt')
for k in a.keys():
  print(a[k].shape)

torch.Size([200091, 96])
torch.Size([200091, 96, 2])
torch.Size([200091, 384])


In [6]:
a['token_start_offsets'].shape, b['token_start_offsets'].shape

NameError: name 'b' is not defined

In [7]:
b = torch.load('news2007_tok96_char384_enc_no_tok.pt')
b.keys()

dict_keys(['token_start_offsets', 'chars_encoded'])

In [8]:
a['chars_encoded'].shape

torch.Size([200091, 384])

In [9]:
a['chars_encoded'].shape[0]/b['chars_encoded'].shape[0]

0.19511365076888573

In [5]:
val_loader = DataLoader(ds[1], batch_size=32, shuffle=False)

In [6]:
idx_to_char_map = {v:k for k,v in torch.load('../data/char_to_idx_map2.pt').items()}

In [7]:
idx_to_char_map[0] = '_'

In [8]:
Torch2Py = lambda x: x.cpu().numpy().tolist()

In [9]:
def bin_acc_fn(outputs, labels):
  binary = outputs>.5
  right = binary==labels
  return right.float()

BCE_loss = nn.BCELoss(reduction='none')
def bce_loss_fn(outputs, labels):
  return BCE_loss(outputs, labels)

def Validate(val_loader, model):
  device = next(model.parameters()).device
  cum_acc = 0
  cum_bce_loss = 0

  inputs_ = []
  losses_ = []
  guesses_ = []
  rights_ = []
  labels_ = []
  for i, data in enumerate(val_loader):
    data = {k: d.to(device) for k,d in data.items()}
    inputs = data['char_encoded_seqs']
    inputs_ += Torch2Py(inputs)
    labels = data['token_start_positions']
    labels_ += Torch2Py(labels)

    with torch.no_grad():
      outputs = model(inputs)
      guesses_ += Torch2Py(outputs)
      bce_loss = bce_loss_fn(outputs, labels)
      losses_ += Torch2Py(bce_loss)
      cum_bce_loss += bce_loss.mean()

      acc = bin_acc_fn(outputs, labels)
      rights_ += Torch2Py(acc)
      cum_acc += acc.mean()

  print(cum_acc/(i+1), cum_bce_loss/(i+1))
  return inputs_, losses_, rights_, labels_, guesses_

In [10]:
inputs, losses, rights, labels, guesses = Validate(val_loader, model)

tensor(0.9887, device='cuda:0') tensor(0.0449, device='cuda:0')


In [11]:
def GetChars(encoded):
  s = ""
  for c in encoded:
    s+= idx_to_char_map[c]
  return s.replace('\x00','_')

In [12]:
c = GetChars(inputs[1])

In [13]:
c

"[cls] you're anyone else, i say just skip it. i can see that this films is mostly rated and commented by girls under 15. no one else could say one phrase to this rubbish. the movie was (as usual) higly boring, stiffly acted, predictable, like a big musicvideo. i simply can_t believe how anyone can write this unpleasant film___________________________________________________________"

In [14]:
s = ""
for i, l in enumerate(labels[1]):
  if l:
    s+= " " + c[i]
print(s)

 [ y ' r a e , i s j s i . i c s t t f i m r a c b g u 1 . n o e c s o p t t r . t m w ( a u ) h g b , s a , p , l a b m v e . i s c _ t b h a c w t u f


In [15]:
length = 100
for i, r in enumerate(rights[:30]):
  if sum(r)<len(r) and i%10==0:
    # underscore above the letter if a tok start was missed
    print("".join('_' if labels[i][j]==1 and x!=1 else ' ' for j, x in enumerate(rights[i][:length])))
    print("".join(str(int(guesses[i][j]*10)) if x!=1 else ' ' for j, x in enumerate(rights[i][:length])))    
    print(GetChars(inputs[i])[:length])
    print("".join(' ' if x==1 else 'X' for x in rights[i][:length]))
    print()

                                                                                                    
                                                                                                    
[cls] called tam lin (one of my favorites), long story short, a woman falls in love with a faerie, a
                                                                                                    

                                                                                                    
                                                                                                    
[cls] person of the same name. his sword can cut anything (except devil's tongue jelly, as i've hear
                                                                                                    



In [16]:
inps = torch.tensor([[32, 46, 55, 62, 33,  1,  3, 46, 15,  1, 63, 51, 58, 56, 44, 62,  3,  1,
         56, 58, 65, 52, 48, 62,  1, 46, 58, 56, 48,  1, 58, 64, 63,  1, 44, 57,
         47,  1, 63, 51, 48, 68,  1, 54, 48, 48, 59,  1, 36, 34,  1, 58, 49,  1,
         63, 51, 48, 56,  1, 52, 57,  1, 32, 56, 33, 15,  1, 62, 58,  1, 47, 58,
         57,  8, 63,  1, 45, 48,  1, 49, 58, 58, 55, 48, 47,  1, 45, 68,  1, 63,
         51, 48,  1,  3, 66, 44, 61,  1, 58, 49,  1, 63, 51, 48,  1, 32, 56, 33,
          3,  1, 52, 57,  1, 68, 58, 64, 61,  1, 56, 58, 65, 52, 48,  1, 62, 63,
         58, 61, 48,  1, 61, 52, 50, 51, 63,  1, 57, 58, 66, 13,  1, 52, 63,  8,
         62,  1, 44,  1, 45,  1, 56, 58, 65, 52, 48, 13,  1, 57, 58, 63,  1, 63,
         58, 56,  1, 46, 61, 64, 52, 62, 48, 15,  1, 66, 58, 66,  2,  1, 63, 51,
         52, 62,  1, 49, 52, 55, 56,  1, 52, 62,  1, 63, 51, 48,  1, 54, 52, 57,
         47,  1, 58, 49,  1, 56, 58, 65, 52, 48,  1, 63, 51, 44, 63,  1, 45, 61,
         52, 57, 50, 62,  1, 63, 48, 44, 61, 62,  1, 63, 58,  1, 68, 58, 64, 61,
          1, 48, 68, 48, 62,  1, 44, 57, 47,  1, 66, 44, 61, 56, 63, 51,  1, 63,
         58,  1, 68, 58, 64, 61,  1, 51, 48, 44, 61, 63, 15,  1, 48, 65, 48, 61,
         68, 58, 57, 48,  1, 46, 58, 57, 57, 48, 46, 63, 48, 47,  1, 63, 58,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0]])

In [17]:
model.tokens_start_attn(inps.cuda())

tensor([[  5.0432,  -9.6778,  -9.4772,  -8.0009,  -6.5148,  -6.6421,   5.0432,
           5.0432,   5.0432,  -6.1596,   5.0432,  -8.1841,  -6.0874,  -3.6117,
          -3.6587,  -3.1781,   5.0432,  -5.1844,   5.0432,  -8.3513,  -6.5491,
          -7.0170,  -5.1324,  -4.3156,  -5.4401,   5.0432,  -8.8702,  -9.8983,
          -5.9861,  -6.6557,   5.0432,  -8.0304,  -6.3259,  -6.1351,   5.0432,
         -10.3148,  -7.6387,  -5.9214,   5.0432,  -7.8010,  -7.5255,  -4.4341,
          -6.5001,   5.0432,  -8.8810,  -4.1256,  -3.8710,  -6.5042,   5.0432,
          -5.9635,  -5.0427,   5.0432,  -8.2271,  -6.1824,   5.0432, -10.1565,
          -7.1697,  -4.2351,  -6.5331,   5.0432,  -7.7881,  -6.6663,   5.0432,
           5.0432,  -1.4903,   5.0432,  -6.1073,   5.0432,  -6.7565,  -5.9914,
           5.0432,  -8.4409,  -6.7468,   5.0432,   5.0432,  -5.5495,   5.0432,
          -6.9397,  -5.4059,   5.0432,  -6.4460,  -3.8810,  -3.7247,  -3.6886,
          -3.8050,  -5.8272,   5.0432,  -6.6779,  -6

In [18]:
chrs = []
for i in inputs:
  chrs.append(GetChars(i))

In [19]:
import plotly.figure_factory as ff
hm = ff.create_annotated_heatmap(
  z=guesses[:12], annotation_text=chrs[:12],
  text=guesses[:12], hoverinfo='text')

plotly.offline.plot(hm, filename='/home/tucker/Downloads/strategy_heatmap.html')


'/home/tucker/Downloads/strategy_heatmap.html'